# Introduction
The previous stage includes reviews text preprocessing, which is in Preprocess_Reviews.ipynb<br>
The following stage is aim to analysis reviews sentiment

In [146]:
import numpy as np
import pandas as pd
import nltk
import seaborn as sns
import re
import spacy
nlp = spacy.load('en_core_web_sm')
import string
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import sys
import os
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" 

import matplotlib.pyplot as plt
%matplotlib inline

# Load reviews

In [147]:
file = "Cleaned_Text_Dataset.csv"
df = pd.read_csv(file)
del df["Unnamed: 0"]
df.head()

,review_id,business_id,text,target,cleaned_text
0,DBlPGlx7mGITUzvtz9A7QQ,n_k7VjsTWjqACsvOihHQHQ,"uninpressed, uninpressive, disapointed, disapo...",0.0,uninpressed uninpressive disapointed disapoint...
1,khHfRuVufqXxbl2KpfAFmA,5l67axuAa5EmEV3lLuSa1Q,I was not impressed with Smashburger. I figure...,0.0,impressed smashburger figure try close apartme...
2,2EcJZtfg_Kko7EIEPkBTTw,W7S5xV7aOIWjcq0Xd4KKLg,Wanted to charge me for an oil change that was...,0.0,want charge oil change suppose free certified ...
3,fm5S-0puAjpCq5idVszLjg,_dZd3_GrQNPNKuP5Eame-Q,This was my first visit to Stacks. Met my fri...,0.0,visit stacks met friend little run door open m...
4,KtGMc-m6DwMLce2IahyvPA,CkENBnSJFfPA1hY0qrqfCg,We have tried this restaurant several times in...,0.0,try restaurant time hope fabulous santa barbar...


为了方便接下来的使用，我需要将dataframe里cleaned_text转为token list

In [148]:
"""
def list_of_token(text):
    return [w for w in text.split()]
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: list_of_token(text))
df
"""

"\ndef list_of_token(text):\n    return [w for w in text.split()]\ndf['cleaned_text'] = df['cleaned_text'].apply(lambda text: list_of_token(text))\ndf\n"

In [149]:
X = df['cleaned_text'] 
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify=y)
X_train.head()

59      star solely base pleasant housekeeper good bri...
6399    great concept food choice great service little...
6210    stop night outside boise cross country trip wa...
1688    manager reach speak phone week hear find new m...
4865    britney great atmosphere lot fun people enjoy ...
Name: cleaned_text, dtype: object

In [150]:
y_train.head()

59      0.0
6399    1.0
6210    1.0
1688    0.0
4865    1.0
Name: target, dtype: float64

建立评估function，它可以生成confusion matrix的heatmap，直观

In [151]:
def evaluate(y_test, predictions):
    cf_matrix = confusion_matrix(y_test, predictions)
    sns.heatmap(cf_matrix, annot = True, fmt = 'd',cmap="Blues")
    plt.title('Heatmap of confusion matrix for Test data')
    plt.ylabel('True label')
    plt.xlabel('Predicted label') 

# Vectorisation and Validation

列举我们将要实验的n-gram，【摘抄】GridSearchCV是Sklearn model_selection包的一个模块，用于超参数调整。 给定一组不同的超参数，GridSearchCV 循环浏览所有可能的超参数值和组合，并在训练数据集上拟合模型。 在这个过程中，它能够确定产生最佳精度的超参数的最佳值和组合（从给定的参数集中）【摘抄】在机器学习模型中，需要人工选择的参数称为超参数。比如随机森林中决策树的个数，人工神经网络模型中隐藏层层数和每层的节点个数，正则项中常数大小等等，他们都需要事先指定。超参数选择不恰当，就会出现欠拟合或者过拟合的问题。而在选择超参数的时候，有两个途径，一个是凭经验微调，另一个就是选择不同大小的参数，带入模型中，挑选表现最好的参数。微调的一种方法是手工调制超参数，直到找到一个好的超参数组合，这么做的话会非常冗长，你也可能没有时间探索多种组合，所以可以使用Scikit-Learn的GridSearchCV来做这项搜索工作。<br>
这里用的是后者<br>
可以提一下交叉验证，cross validation

In [152]:
param_grid = {'c_vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]}

### Logistic Regression model 
找出哪个n-gram在逻辑回归模型中表现更好

In [153]:
lr_pipeline = Pipeline([
    ('c_vectorizer', CountVectorizer()),
    ('lr', LogisticRegression(random_state=42))
])

gs_lr = GridSearchCV(lr_pipeline, refit=True, cv=2, param_grid=param_grid, scoring='f1', n_jobs=-1)
gs_lr.fit(X_train, y_train)

print('Optimal n-gram:', gs_lr.best_estimator_.get_params()['c_vectorizer__ngram_range'])
print("最优参数: ", gs_lr.best_params_)
print("最佳性能: ", gs_lr.best_score_)

predictions = gs_lr.predict(X_test)
print(classification_report(y_test, predictions, digits=4))
#evaluate(y_test, predictions)

Optimal n-gram: (1, 2)
最优参数:  {'c_vectorizer__ngram_range': (1, 2)}
最佳性能:  0.9322097128967155
              precision    recall  f1-score   support

         0.0     0.9460    0.9881    0.9666      1011
         1.0     0.9876    0.9436    0.9651      1011

    accuracy                         0.9659      2022
   macro avg     0.9668    0.9659    0.9659      2022
weighted avg     0.9668    0.9659    0.9659      2022



### Support Vector Machine model 
找出哪个n-gram在支持向量机SVM模型中表现更好

In [157]:
svm_pipe = Pipeline([
    ('c_vectorizer', CountVectorizer()),
    ('svm', svm.SVC(max_iter=-1, random_state=42))
])

gs_svm = GridSearchCV(svm_pipe, refit=True, cv=2, param_grid=param_grid, scoring='f1', n_jobs=-1)
gs_svm.fit(X_train, y_train)

print('Best ngram_range:', gs_svm.best_estimator_.get_params()['c_vectorizer__ngram_range'])
print("最优参数: ", gs_svm.best_params_)
print("最佳性能: ", gs_svm.best_score_)

predictions = gs_svm.predict(X_test)
print(classification_report(y_test, predictions, digits=4))
#evaluate(y_test, predictions)

Best ngram_range: (1, 3)
最优参数:  {'c_vectorizer__ngram_range': (1, 3)}
最佳性能:  0.9208088884488121
              precision    recall  f1-score   support

         0.0     0.9574    0.9773    0.9672      1011
         1.0     0.9768    0.9565    0.9665      1011

    accuracy                         0.9669      2022
   macro avg     0.9671    0.9669    0.9669      2022
weighted avg     0.9671    0.9669    0.9669      2022



对比上面逻辑回归和SVM交叉验证的结果，逻辑回归的最佳性能更好，因此我们选择最佳性能更好的逻辑回归结果，它的最优参数n-gram是（1，2）<br>接下来使用逻辑回归和

### Using TF_IDF
【摘抄】we can use TF_IDF vectorizing to find the weighted words that occur more frequently in the document that leads to creation of the bag of words model我们可以使用 TF_IDF 向量化来找到文档中出现频率更高的加权词，从而创建词袋模型